In [1]:
import nltk
from nltk.corpus import wordnet as wn
import spacy
from pprint import pprint
import numpy as np

nltk.download('wordnet')

# Load NER model
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package wordnet to /Users/yyh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load in Questions/Answers
QaA = []

with open('QuestionsAndAnswers.txt', mode='r') as file:
    for line in file:
        q, a = line.strip().split(':/:')

        QaA.append({'question': q, 'answer': a})


# List of named entities
    PERSON:      People, including fictional.
    NORP:        Nationalities or religious or political groups.
    FAC:         Buildings, airports, highways, bridges, etc.
    ORG:         Companies, agencies, institutions, etc.
    GPE:         Countries, cities, states.
    LOC:         Non-GPE locations, mountain ranges, bodies of water.
    PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
    EVENT:       Named hurricanes, battles, wars, sports events, etc.
    WORK_OF_ART: Titles of books, songs, etc.
    LAW:         Named documents made into laws.
    LANGUAGE:    Any named language.
    DATE:        Absolute or relative dates or periods.
    TIME:        Times smaller than a day.
    PERCENT:     Percentage, including ”%“.
    MONEY:       Monetary values, including unit.
    QUANTITY:    Measurements, as of weight or distance.
    ORDINAL:     “first”, “second”, etc.
    CARDINAL:    Numerals that do not fall under another type.

In [3]:
NER = []

for qa in QaA:
    q_doc, a_doc = nlp(qa['question']), nlp(qa['answer'])
    
    dont_include_labels = ['DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']

    print([q_doc]+[(entity.text, entity.label_) for entity in q_doc.ents if entity.label_ not in dont_include_labels])
    print([a_doc]+[(entity.text, entity.label_) for entity in a_doc.ents if entity.label_ not in dont_include_labels])

[Is it correct that Berlin is the capital of Germany?, ('Berlin', 'GPE'), ('Germany', 'GPE')]
[surely it should be Bonn? Quote from: Snoopy on December 09, 2017, 12:3, ('Bonn', 'GPE'), ('Snoopy', 'PERSON')]
[Does Japan have the highest life expectancy in the world?, ('Japan', 'GPE')]
[февруари 30, 2017 A study by a team of researchers from the Institute for Health Metrics and E, ('the Institute for Health Metrics', 'ORG')]
[True or False: Brazil’s official language is Portuguese., ('Brazil', 'GPE'), ('Portuguese', 'NORP')]
[Hinweis: Diese Aussage ist falsch! True or False: It is possible to take a bus from Cuiaba to Brasilia. Hinweis:, ('Hinweis', 'PERSON'), ('Diese Aussage', 'NORP'), ('Cuiaba', 'GPE'), ('Brasilia', 'GPE'), ('Hinweis', 'PERSON')]
[Would it be accurate to say the Eiffel Tower is in Paris?, ('the Eiffel Tower', 'LOC'), ('Paris', 'GPE')]
[surely. Would it be acceptable to say that the Eiffel tower is in Paris, France or in Paris, France? What you have, ('Eiffel', 'LOC'), 

In [ ]:
import requests

def generate_candidates_api(mention, language="en", limit=10):
    url = "https://www.wikidata.org/w/api.php"

    params = {
        "action": "wbsearchentities",
        "search": mention,
        "language": language,
        "format": "json",
        "limit": limit,
    }

    response = requests.get(url, params=params)
    data = response.json()
    
    candidates = []

    for entity in data.get("search", []):
        candidates.append(entity["id"])
    return candidates



def get_entity_info(id, languages='en'):

    url = "https://www.wikidata.org/w/api.php"

    params = {
        "action": "wbgetentities",
        "ids": id,
        "languages": languages,
        "format": "json",
    }

    response = requests.get(url, params=params)
    data = response.json()

    label = data['entities'][id]['labels']['en']['value']
    description = data['entities'][id]['descriptions']['en']['value']
    claims = len(data['entities'][id]['claims'].keys())
    sitelinks =  len(data['entities'][id]['sitelinks'].keys())
    
    url = ''

    if data['entities'][id]['sitelinks'].get('enwiki'):
        base_url = 'https://en.wikipedia.org/wiki/'
        url = base_url + data['entities'][id]['sitelinks']['enwiki']['title'].replace(' ', '_')

    return {'label': label, 'description': description, 'claims': claims, 'sitelinks': sitelinks, 'url': url}

In [14]:
query = "Homer"

candidates = generate_candidates_api(query)

for id in candidates:
    print(get_entity_info(id))

{'label': 'Homer', 'description': 'male given name', 'claims': 11, 'sitelinks': 2, 'url': ''}
{'label': 'Homer', 'description': 'family name', 'claims': 12, 'sitelinks': 1, 'url': ''}
{'label': 'Homer', 'description': 'reputed author of the Iliad and the Odyssey', 'claims': 313, 'sitelinks': 265, 'url': 'https://en.wikipedia.org/wiki/Homer'}
{'label': 'Winslow Homer', 'description': 'American landscape painter (1836–1910)', 'claims': 166, 'sitelinks': 43, 'url': 'https://en.wikipedia.org/wiki/Winslow_Homer'}
{'label': 'Homer Simpson', 'description': 'character from The Simpsons franchise', 'claims': 67, 'sitelinks': 89, 'url': 'https://en.wikipedia.org/wiki/Homer_Simpson'}
{'label': 'Homer', 'description': 'name', 'claims': 8, 'sitelinks': 2, 'url': 'https://en.wikipedia.org/wiki/Homer_(name)'}
{'label': 'Homer', 'description': 'city\xa0in\xa0Kenai Peninsula Borough, Alaska, United States', 'claims': 43, 'sitelinks': 54, 'url': 'https://en.wikipedia.org/wiki/Homer,_Alaska'}
{'label': '